In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
!pip install tmdbsimple
import tmdbsimple as tmdb
import os, time,json

# Part 1 

# - Importing the Data

In [2]:
basics_url='Data/title_basics_cleaned.csv.gz'
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [3]:
ratings_url='Data/title_ratings_cleaned.csv.gz'
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [4]:
akas_url='Data/title_akas_cleaned.csv.gz'
df_reader = pd.read_csv(akas_url, sep='\t', low_memory=False, chunksize=100_000)
df_reader

In [5]:
df_reader._currow

0

In [6]:
temp_df = df_reader.get_chunk()
temp_df

,"titleId,ordering,title,region,language,types,attributes,isOriginalTitle"
0,"tt0000001,6,Carmencita,US,,imdbDisplay,,0"
1,"tt0000002,7,The Clown and His Dogs,US,,,litera..."
2,"tt0000005,10,Blacksmith Scene,US,,imdbDisplay,,0"
3,"tt0000005,1,Blacksmithing Scene,US,,alternativ..."
4,"tt0000005,6,Blacksmith Scene #1,US,,alternativ..."
...,...
99995,"tt0121748,2,Short,US,,imdbDisplay,,0"
99996,"tt0121750,5,Four Dragons,US,,imdbDisplay,,0"
99997,"tt0121755,3,A Single Spark,US,,imdbDisplay,,0"
99998,"tt0121756,2,Six Point Nine,US,,imdbDisplay,,0"


In [7]:
df_reader._currow

100000

In [14]:
temp_df.replace({'\\N':np.nan},inplace=True)
temp_df = temp_df[(temp_df['region'] == 'US')]
temp_df

KeyError: 'region'

In [9]:
chunk_num=1
fname= f'Data/title_akas_chunk_{chunk_num:03d}.csv.gz'
fname

'Data/title_akas_chunk_001.csv.gz'

In [10]:
## Save temp_df to disk using the fname.
temp_df.to_csv(fname, compression='gzip')
## incrementing chunk_num by 1 for the next file.
chunk_num+=1

In [11]:
pd.read_csv(fname)

,Unnamed: 0,"titleId,ordering,title,region,language,types,attributes,isOriginalTitle"


In [12]:
 
akas_url = 'Data/title_akas_cleaned.csv.gz'
chunk_num = 1
df_reader = pd.read_csv(akas_url, sep='\t',
                        low_memory=False, chunksize=100_000)

In [13]:
for temp_df in df_reader:
        #### COMBINED WORKFLOW FROM ABOVE
    ## Replace "\N" with np.nan
    temp_df.replace({'\\N':np.nan},inplace=True)
    temp_df = temp_df[(temp_df['titleId,ordering,title,region,language,types,attributes,isOriginalTitle'] == 'US')]
     
    ### Saving chunk to disk
    fname= f'title_akas_chunk_{chunk_num:03d}.csv.gz'
    temp_df.to_csv(fname, compression='gzip')
    print(f"- Saved {fname}")
    
    ## increment chunk_num    
    chunk_num+=1
## Closing the reader now that we are done looping through the file
df_reader.close()

- Saved title_akas_chunk_001.csv.gz
- Saved title_akas_chunk_002.csv.gz
- Saved title_akas_chunk_003.csv.gz
- Saved title_akas_chunk_004.csv.gz
- Saved title_akas_chunk_005.csv.gz
- Saved title_akas_chunk_006.csv.gz
- Saved title_akas_chunk_007.csv.gz
- Saved title_akas_chunk_008.csv.gz
- Saved title_akas_chunk_009.csv.gz
- Saved title_akas_chunk_010.csv.gz


ParserError: Error tokenizing data. C error: Expected 1 fields in line 1068060, saw 6


In [ ]:
import glob
q = "title_akas_chunk*.csv.gz"
chunked_files = glob.glob(q)
# Showing the first 5
chunked_files[:5]

In [ ]:
q = "title_akas_chunk*.csv.gz"
chunked_files = sorted(glob.glob(q))
# Showing the first 5 
chunked_files[:5]

In [ ]:
# Recursive query - extra /**/ added to string
q = "**/title_akas_chunk*.csv.gz" 
chunked_files = sorted(glob.glob(q, recursive=True)) 
# Showing the first 5 
chunked_files[:5]

In [ ]:
## Loading all files as df and appending to a list
df_list = []
for file in chunked_files:
    temp_df = pd.read_csv(file, index_col=0)
    df_list.append(temp_df)
    
## Concatenating the list of dfs into 1 combined
df_combined = pd.concat(df_list)
df_combined

In [ ]:
## Saving the final combined dataframe
final_fname ='title_akas_combined.csv.gz'
df_combined.to_csv(final_fname, compression='gzip', index=False)

# Filtering/ Cleaning
1. Title Basics

In [ ]:
basics.replace({'\\N':np.nan},inplace=True)

In [ ]:
basics.head()

In [ ]:
basics = basics.dropna(subset=["tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres"])

In [ ]:
basics = basics.dropna(subset=["tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres"])

In [ ]:
basics = basics.loc[(basics['tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres']=='movies')] 
basics

In [ ]:
basics = basics[(basics['tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres']>=2000)&(basics['tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres']<2022)]
basics

In [ ]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres'].str.contains('tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres',case=False)
basics_df = basics[~is_documentary]


In [ ]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres'].isin(basics['tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres'])
keepers

In [ ]:
basics = basics[keepers]
basics

In [ ]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

In [ ]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [ ]:
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()